## CT and RT Structure data Pre-processing for multi-label survival prediction using planing CT

### 1. dicom to nii conversion

In [6]:
from dcmrtstruct2nii import dcmrtstruct2nii, list_rt_structs
import os, sys, glob

from tqdm.notebook import tqdm
from time import sleep

In [7]:
oriPath = '../../data/OPC_Radiomics/OPC-Radiomics/' # can be download at https://wiki.cancerimagingarchive.net/pages/viewpage.action?pageId=33948764
# we have download it to  T:\Physics_Research\LABS\LAB_Wang_Zhang\Data\OPC_Radiomics
patients = os.listdir(oriPath)

savePath = '../../data/OPC_Radiomics/OPC-Radiomics-Nii/' # wherever you want to save the Nii files
if not os.path.exists(savePath):
   os.makedirs(savePath)

In [11]:
# for i in tqdm(range(len(patients))):
#     sleep(0.01)
    # p = patients[i]
for p in patients:
    folder = os.listdir(os.path.join(oriPath, p))
    if len(folder)==0:
        print('!!!!! No data for ',p)
        continue
    subFolder = os.listdir(os.path.join(oriPath, p, folder[0]))
    if len(subFolder)<2:
        print('!!!!! No engough data for ',p)
        continue
    pSavePath = os.path.join(savePath,p)

    RTfile = None
    imagePath = None

    for s in subFolder:
        if len(os.listdir(os.path.join(oriPath, p, folder[0], s)))==1:
            RTfile = glob.glob(os.path.join(oriPath, p, folder[0], s,'00000*'))
        else:
            imagePath = os.path.join(oriPath, p, folder[0], s)
    # dcmrtstruct2nii(RTfile[0],imagePath,pSavePath)
    try:
        dcmrtstruct2nii(RTfile[0],imagePath,pSavePath)
        print('Img and RTStructure Niis Generated for ', p)
    except:
        print('!!!!!!!!!!!!!!! Something wrong with ', p)

    # print(RTfile[0])
    # print(imagePath)
    # print(pSavePath)

!!!!! No engough data for  OPC-00312
Img and RTStructure Niis Generated for  OPC-00260
Img and RTStructure Niis Generated for  OPC-00182
Img and RTStructure Niis Generated for  OPC-00001
Img and RTStructure Niis Generated for  OPC-00374
Img and RTStructure Niis Generated for  OPC-00317
Img and RTStructure Niis Generated for  OPC-00360
Img and RTStructure Niis Generated for  OPC-00517
Img and RTStructure Niis Generated for  OPC-00187
Img and RTStructure Niis Generated for  OPC-00506
Img and RTStructure Niis Generated for  OPC-00313
Img and RTStructure Niis Generated for  OPC-00212
Img and RTStructure Niis Generated for  OPC-00305
Img and RTStructure Niis Generated for  OPC-00218
Img and RTStructure Niis Generated for  OPC-00419
Img and RTStructure Niis Generated for  OPC-00405
Img and RTStructure Niis Generated for  OPC-00008
Img and RTStructure Niis Generated for  OPC-00325
Img and RTStructure Niis Generated for  OPC-00063
Img and RTStructure Niis Generated for  OPC-00281
Img and RTStr

In [15]:
savedPatients = os.listdir(savePath)
len(savedPatients)

597

### 2. Figure out how many patient data are available

In [18]:
missing_gtv = []
missing_img = []
for p in savedPatients:
    path = os.path.join(savePath, p)
    files = os.listdir(path)

    if 'image.nii.gz' in files:
        if 'mask_GTV.nii.gz' in files:
            continue
        else:
            missing_gtv.append(p)
            print('This patient dosen\'t have GTV', p)
            
    else:
        missing_img.append(p)
        print('This patient dosen\'t have img', p)

This patient dosen't have GTV OPC-00502
This patient dosen't have GTV OPC-00522
This patient dosen't have GTV OPC-00094
This patient dosen't have GTV OPC-00478
This patient dosen't have GTV OPC-00175
This patient dosen't have GTV OPC-00484
This patient dosen't have GTV OPC-00278
This patient dosen't have GTV OPC-00365
This patient dosen't have GTV OPC-00221
This patient dosen't have GTV OPC-00206
This patient dosen't have GTV OPC-00183
This patient dosen't have GTV OPC-00095
This patient dosen't have GTV OPC-00374
This patient dosen't have GTV OPC-00187
This patient dosen't have GTV OPC-00571
This patient dosen't have GTV OPC-00457
This patient dosen't have GTV OPC-00479
This patient dosen't have GTV OPC-00342
This patient dosen't have GTV OPC-00430
This patient dosen't have GTV OPC-00101
This patient dosen't have GTV OPC-00191
This patient dosen't have GTV OPC-00340
This patient dosen't have GTV OPC-00330
This patient dosen't have GTV OPC-00220
This patient dosen't have GTV OPC-00369


In [19]:
len(missing_gtv) # actually, for most of these patients, they have data but they don't have gtvp, we can use empty image (0 image) for them

85

In [20]:
len(missing_img)

0

In [21]:
avaliable_patient = list(set(savedPatients) - set(missing_gtv))
len(avaliable_patient)

512

### 3. Resampling

In [23]:
from pathlib import Path
from multiprocessing import Pool
import logging

import click
import pandas as pd
import numpy as np
import SimpleITK as sitk

In [24]:
resample_path = '../../data/OPC_Radiomics/OPC-Radiomics-Nii-resample/'
if not os.path.exists(resample_path):
   os.makedirs(resample_path)

In [27]:
resampler = sitk.ResampleImageFilter()
resampler.SetOutputDirection([1, 0, 0, 0, 1, 0, 0, 0, 1])
resampling = [2,2,2] # mm
resampler.SetOutputSpacing(resampling)

In [28]:
def get_bouding_boxes(ct, pt):
    """
    Get the bounding boxes of the CT and PT images.
    This works since all images have the same direction
    """

    ct_origin = np.array(ct.GetOrigin())
    pt_origin = np.array(pt.GetOrigin())

    ct_position_max = ct_origin + np.array(ct.GetSize()) * np.array(
        ct.GetSpacing())
    pt_position_max = pt_origin + np.array(pt.GetSize()) * np.array(
        pt.GetSpacing())
    return np.concatenate(
        [
            np.maximum(ct_origin, pt_origin),
            np.minimum(ct_position_max, pt_position_max),
        ],
        axis=0,
    )

In [29]:
def resample_one_patient(p):
    savePath = '../../data/OPC_Radiomics/OPC-Radiomics-Nii/' # change it to wherever you need
    resample_path = '../../data/OPC_Radiomics/OPC-Radiomics-Nii-resample/' # change it to wherever you need to save the resampled data
    
    ct = sitk.ReadImage(os.path.join(savePath, p, 'image.nii.gz'))
    label = sitk.ReadImage(os.path.join(savePath, p, 'mask_GTV.nii.gz'))
    bb = get_bouding_boxes(ct, ct)
    size = np.round((bb[3:] - bb[:3]) / resampling).astype(int)
    resampler.SetOutputOrigin(bb[:3])
    resampler.SetSize([int(k) for k in size])  # sitk is so stupid
    resampler.SetInterpolator(sitk.sitkBSpline)
    ct = resampler.Execute(ct)
    sitk.WriteImage(ct, os.path.join(resample_path, p+'_image.nii.gz'))
    resampler.SetInterpolator(sitk.sitkNearestNeighbor)
    label = resampler.Execute(label)
    sitk.WriteImage(label, os.path.join(resample_path, p+'_mask_GTV.nii.gz'))

In [30]:
for p in avaliable_patient:
    resample_one_patient(p)

### 4. Cropping

In [31]:
def find_centroid(mask):

    stats = sitk.LabelShapeStatisticsImageFilter()
    stats.Execute(mask)
    try:
        centroid_coords = stats.GetCentroid(255)
    except:
        print('Something wrong')
    centroid_idx = mask.TransformPhysicalPointToIndex(centroid_coords)

    return np.asarray(centroid_idx, dtype=np.float64)

In [32]:
crop_path = '../../data/OPC_Radiomics/OPC-Radiomics-Nii-resample-crop/' # the final data you are going to use in our model
if not os.path.exists(crop_path):
   os.makedirs(crop_path)

In [35]:
for p in avaliable_patient:
    
    image = sitk.ReadImage(os.path.join(resample_path, p+'_image.nii.gz'))
    mask = sitk.ReadImage(os.path.join(resample_path, p+'_mask_GTV.nii.gz'))

    patch_size = np.array([80,80,48])

    #crop the image to patch_size around the tumor center
    tumour_center = find_centroid(mask) # center of GTV
    size = patch_size
    min_coords = np.floor(tumour_center - size / 2).astype(np.int64)
    max_coords = np.floor(tumour_center + size / 2).astype(np.int64)
    min_x, min_y, min_z = min_coords
    max_x, max_y, max_z = max_coords
    image = image[min_x:max_x, min_y:max_y, min_z:max_z]

    # window image intensities to [-500, 1000] HU range
    image = sitk.Clamp(image, sitk.sitkFloat32, -500, 500)
    mask = mask[min_x:max_x, min_y:max_y, min_z:max_z]

    sitk.WriteImage(image, os.path.join(crop_path, p+'_image.nii.gz'))
    sitk.WriteImage(mask, os.path.join(crop_path, p+'_mask_GTV.nii.gz'))

In [37]:
import csv

with open('avaliable_patient.csv','w') as result_file:
    wr = csv.writer(result_file, dialect='excel')
    for p in avaliable_patient:
        wr.writerow([p])